In [2]:
import os
print (os.environ['CONDA_DEFAULT_ENV'])

general_venv2


### Recall Goal of Problem: predict up to 5 hotel clusters to recommend to users

Note: clusters are determined via an internal algorithm not exposed to us

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [39]:
DATA_FOLDER = 'data/'
CSV_EXTENSION = '.csv'
SRCH_DEST_ID = 'srch_destination_id'

## Read in Data

In [9]:
! ls data/

destinations.csv
expedia-hotel-recommendations.zip
sample_submission.csv
test.csv
train.csv


In [11]:
train = pd.read_csv(DATA_FOLDER+'train'+CSV_EXTENSION)
train.head()

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
0,2014-08-11 07:46:59,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,0,3,2,50,628,1
1,2014-08-11 08:22:12,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,1,1,2,50,628,1
2,2014-08-11 08:24:33,2,3,66,348,48862,2234.2641,12,0,0,...,0,1,8250,1,0,1,2,50,628,1
3,2014-08-09 18:05:16,2,3,66,442,35390,913.1932,93,0,0,...,0,1,14984,1,0,1,2,50,1457,80
4,2014-08-09 18:08:18,2,3,66,442,35390,913.6259,93,0,0,...,0,1,14984,1,0,1,2,50,1457,21


In [10]:
destinations = pd.read_csv(DATA_FOLDER+'destinations'+CSV_EXTENSION)
destinations.head()

,srch_destination_id,d1,d2,d3,d4,d5,d6,d7,d8,d9,...,d140,d141,d142,d143,d144,d145,d146,d147,d148,d149
0,0,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-1.897627,-2.198657,-2.198657,-1.897627,...,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657,-2.198657
1,1,-2.181690,-2.181690,-2.181690,-2.082564,-2.181690,-2.165028,-2.181690,-2.181690,-2.031597,...,-2.165028,-2.181690,-2.165028,-2.181690,-2.181690,-2.165028,-2.181690,-2.181690,-2.181690,-2.181690
2,2,-2.183490,-2.224164,-2.224164,-2.189562,-2.105819,-2.075407,-2.224164,-2.118483,-2.140393,...,-2.224164,-2.224164,-2.196379,-2.224164,-2.192009,-2.224164,-2.224164,-2.224164,-2.224164,-2.057548
3,3,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.115485,-2.177409,-2.177409,-2.177409,...,-2.161081,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409,-2.177409
4,4,-2.189562,-2.187783,-2.194008,-2.171153,-2.152303,-2.056618,-2.194008,-2.194008,-2.145911,...,-2.187356,-2.194008,-2.191779,-2.194008,-2.194008,-2.185161,-2.194008,-2.194008,-2.194008,-2.188037


<font color = 'magenta'> Note: the destinations csv joins to train csv on destination.srch_destination_id == train.srch_destination_id; would join, but destinations.csv has 149 columns. We should proabably join when actually training </font>

### to search:
- type of each column (float, int, cat/ord)
- distributions of each (min,  max, boxplots maybe?)
- colinear features?
- any NaNs?
- plots of clusters?


## Feature Types

screenshots grabbed from kaggle data description: [https://www.kaggle.com/c/expedia-hotel-recommendations/data?select=sample_submission.csv](https://www.kaggle.com/c/expedia-hotel-recommendations/data?select=sample_submission.csv)
![data-type-1](imgs/data-type-1.png)
![data-type-2](imgs/data-type-2.png)

<font color='blue'> 
    All the features are either an int or float, however upon closer inspection they all appear to be a categorical feature. The ones that are not are listed below:
    - orig_destination_distance
    - srch_ci
    - srch_co
    - srch_adults_cnt
    - srch_children_cnt
    - cnt
</font>

<br>
<br>
<font color='magenta'>
    We need to consider how to encode these in someway. one hot vs target? I think we should encode these, bc having srch_destination_id = 8250 is not inherently worse than having it = 14984. Also, scaling?. bc ids can get incredibly high
</font>

## NaNs?

In [90]:
def get_cols_with_nas_percent_rows(df):
    res = []
    df_nas = df.isna().sum()/df.shape[0]
    for i, val in df_nas.items():
        if val > 0:
            res.append((i, val))
    return res

In [73]:
def get_percent_cols_with_nas(df):
    cnt = 0
    for i, val in df.isna().sum().items():
        if val > 0:
            cnt += 1
    return cnt/len(df.columns)

In [91]:
print(get_cols_with_nas_percent_rows(train))
print('% cols with nas:', get_percent_cols_with_nas(train) * 100)

[('orig_destination_distance', 0.35903625703150227), ('srch_ci', 0.0012498708199588466), ('srch_co', 0.0012498973660757032)]
% cols with nas: 12.5


<font color='magenta'>
In my opinion, 12.5% is not an insignificant number of columns with NaNs, so we shouldn't drop those column
</font>

<br>
<br>

<font color='magenta'>
Regarding the dropping the pts with NaNs, I'll leave that to @Austin & @Rishav to consider. 35% is a lot of rows and 12% is also not insignificant. But perhaps see if the distribution of hotel cluster would change if dropped those rows
</font>

In [92]:
print(get_cols_with_nas_percent_rows(destinations))
print('% cols with nas:', get_percent_cols_with_nas(destinations) * 100)

[]
% cols with nas: 0.0


## Do we have all categorical ids for each feature?

In [21]:
np.sort(train['site_name'].unique())

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 40, 41, 43, 44, 45, 46, 47, 48, 53], dtype=int64)

<font color='blue'>
    site_name ids do not incrememntally go up in the trian data. Note that it's missing ids 1, 42, and 49 (as well as a few others). Looking at posa_continent's min (which is 0), it is also feasible that site_name ids  is also missing id=0. It is odd for those numbers to be explicitly skipped. It's likely that the entire dataset has pts with those missing ids, but they're not present in our train set. This implies we'll probably have to handle seeing unknown/never-before-seen feature values in the test set.
</font>

In [23]:
np.sort(train['user_location_country'].unique())

array([  0,   1,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
        41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,
        54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
        67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
        93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
       106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
       119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131,
       132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144,
       145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157,
       158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170,
       171, 172, 173, 174, 175, 176, 178, 179, 180, 181, 182, 18

<font color='blue'>
    user_location_country is also missing some ids (like site_name). so the before mentioned problem is probalby going to be an issue accross all categorical features.
</font>

In [25]:
destinations.describe()

,srch_destination_id,d1,d2,d3,d4,d5,d6,d7,d8,d9,...,d140,d141,d142,d143,d144,d145,d146,d147,d148,d149
count,62106.000000,62106.000000,62106.000000,62106.000000,62106.000000,62106.000000,62106.000000,62106.000000,62106.000000,62106.000000,...,62106.000000,62106.000000,62106.000000,62106.000000,62106.000000,62106.000000,62106.000000,62106.000000,62106.000000,62106.000000
mean,32359.463884,-2.193903,-2.202854,-2.207391,-2.194040,-2.161497,-2.045110,-2.202433,-2.203207,-2.107808,...,-2.204092,-2.196919,-2.203262,-2.205128,-2.201925,-2.203332,-2.202989,-2.208359,-2.208269,-2.199470
std,18711.765765,0.038576,0.041065,0.040092,0.041406,0.066197,0.135803,0.038886,0.036578,0.197904,...,0.037164,0.059914,0.046520,0.039465,0.041603,0.038239,0.051552,0.038035,0.038569,0.042438
min,0.000000,-2.597617,-2.671613,-2.671613,-2.671613,-2.671613,-2.344165,-2.671613,-2.671613,-2.495544,...,-2.620769,-2.671613,-2.671613,-2.671613,-2.671613,-2.671613,-2.671613,-2.671613,-2.671613,-2.671613
25%,16320.250000,-2.209336,-2.220192,-2.223679,-2.211437,-2.192158,-2.154267,-2.218745,-2.219086,-2.197704,...,-2.219630,-2.216147,-2.220347,-2.221893,-2.218394,-2.220500,-2.221748,-2.224618,-2.224290,-2.216428
50%,32277.500000,-2.185969,-2.193247,-2.195208,-2.188057,-2.178255,-2.088598,-2.192513,-2.192915,-2.177229,...,-2.192232,-2.190525,-2.192814,-2.194127,-2.191564,-2.192200,-2.193352,-2.196185,-2.195900,-2.190953
75%,48467.750000,-2.175309,-2.179261,-2.179990,-2.176763,-2.153317,-1.975940,-2.178976,-2.179164,-2.123598,...,-2.178093,-2.177777,-2.178996,-2.179475,-2.178335,-2.177590,-2.178927,-2.180602,-2.180380,-2.178088
max,64993.000000,-1.596004,-1.341325,-1.917826,-1.814585,-1.607558,-1.209058,-1.474441,-1.879678,-0.977219,...,-1.855317,-0.960356,-1.357408,-1.775218,-1.790435,-1.799341,-1.335962,-1.816892,-1.718778,-1.500309


In [40]:
srch_dest_set = set(destinations[SRCH_DEST_ID])
srch_dest_id = 0
missing = []
while srch_dest_id <= destinations[SRCH_DEST_ID].max():
    if srch_dest_id not in srch_dest_set:
        missing.append(srch_dest_id)
    srch_dest_id += 1

In [37]:
print(np.array(missing))
print(len(missing))

[   12    22   133 ... 64972 64984 64985]
2888


<font color='blue'>
    Interesting. Would have thought that we would have all ids for srch_destination_id in the destinations csv. Maybe they were distinctly left out? Are there srch_destination_ids in train that don't exist in destiantions? What about the other way around?
    </font>
   

In [89]:
train_srch_dest_ids_set = set(train[SRCH_DEST_ID])
dest_srch_dest_ids_set = set(destinations[SRCH_DEST_ID])

print('in train not in dest:')
train_minus_dest = train_srch_dest_ids_set - dest_srch_dest_ids_set
print(len(train_minus_dest))

print()

print('in dest not in train:')
dest_minus_train = dest_srch_dest_ids_set - train_srch_dest_ids_set
print(len(dest_minus_train))

in train not in dest:
1662

in dest not in train:
4313


<font color='blue'>
    Okay, so apparently it's not 1-1. Data is rarely clean in the real world, so while unfortuante, not unexpected.
</font>
    
<br>
<br>
    
<font color = 'magenta'>
    Poses the question: what do we do now for ids in train, but not dest. should we impute the data? drop those rows?
    </font?

## Distributions of Features

<font color='magenta'>
    Based on what we defined as "insights from explorations" (some plots, relations, scaling", I'll leave this section to @Yinbo and @Junyi. Some (potentially) helpful things to plot: distribution of target variable (hotel_cluster) for each categorical var (maybe side-by-side bar plots?), maybe some sort of cluster plot? idk kinda just spitballing. Maybe take a look at hw1 for inspiration?
</font>

In [14]:
train.describe()

,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,...,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
count,3.767029e+07,3.767029e+07,3.767029e+07,3.767029e+07,3.767029e+07,2.414529e+07,3.767029e+07,3.767029e+07,3.767029e+07,3.767029e+07,...,3.767029e+07,3.767029e+07,3.767029e+07,3.767029e+07,3.767029e+07,3.767029e+07,3.767029e+07,3.767029e+07,3.767029e+07,3.767029e+07
mean,9.795271e+00,2.680473e+00,8.610880e+01,3.084060e+02,2.775304e+04,1.970090e+03,6.044518e+05,1.349265e-01,2.489042e-01,5.870761e+00,...,3.321222e-01,1.112663e+00,1.444109e+04,2.582280e+00,7.965675e-02,1.483384e+00,3.156305e+00,8.129685e+01,6.004619e+02,4.980861e+01
std,1.196754e+01,7.480393e-01,5.924310e+01,2.084437e+02,1.678255e+04,2.232442e+03,3.506175e+05,3.416451e-01,4.323782e-01,3.717095e+00,...,7.314981e-01,4.591155e-01,1.106630e+04,2.153019e+00,2.707611e-01,1.219776e+00,1.623189e+00,5.617119e+01,5.117391e+02,2.891595e+01
min,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.600000e-03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.000000e+00,3.000000e+00,6.600000e+01,1.740000e+02,1.300900e+04,3.131670e+02,2.989100e+05,0.000000e+00,0.000000e+00,2.000000e+00,...,0.000000e+00,1.000000e+00,8.267000e+03,1.000000e+00,0.000000e+00,1.000000e+00,2.000000e+00,5.000000e+01,1.600000e+02,2.500000e+01
50%,2.000000e+00,3.000000e+00,6.600000e+01,3.140000e+02,2.765500e+04,1.140491e+03,6.039140e+05,0.000000e+00,0.000000e+00,9.000000e+00,...,0.000000e+00,1.000000e+00,9.147000e+03,1.000000e+00,0.000000e+00,1.000000e+00,2.000000e+00,5.000000e+01,5.930000e+02,4.900000e+01
75%,1.400000e+01,3.000000e+00,7.000000e+01,3.850000e+02,4.241300e+04,2.552599e+03,9.101680e+05,0.000000e+00,0.000000e+00,9.000000e+00,...,0.000000e+00,1.000000e+00,1.879000e+04,5.000000e+00,0.000000e+00,2.000000e+00,4.000000e+00,1.060000e+02,7.010000e+02,7.300000e+01
max,5.300000e+01,4.000000e+00,2.390000e+02,1.027000e+03,5.650800e+04,1.240790e+04,1.198785e+06,1.000000e+00,1.000000e+00,1.000000e+01,...,9.000000e+00,8.000000e+00,6.510700e+04,9.000000e+00,1.000000e+00,2.690000e+02,6.000000e+00,2.120000e+02,2.117000e+03,9.900000e+01


In [93]:
destinations.describe()

,srch_destination_id,d1,d2,d3,d4,d5,d6,d7,d8,d9,...,d140,d141,d142,d143,d144,d145,d146,d147,d148,d149
count,62106.000000,62106.000000,62106.000000,62106.000000,62106.000000,62106.000000,62106.000000,62106.000000,62106.000000,62106.000000,...,62106.000000,62106.000000,62106.000000,62106.000000,62106.000000,62106.000000,62106.000000,62106.000000,62106.000000,62106.000000
mean,32359.463884,-2.193903,-2.202854,-2.207391,-2.194040,-2.161497,-2.045110,-2.202433,-2.203207,-2.107808,...,-2.204092,-2.196919,-2.203262,-2.205128,-2.201925,-2.203332,-2.202989,-2.208359,-2.208269,-2.199470
std,18711.765765,0.038576,0.041065,0.040092,0.041406,0.066197,0.135803,0.038886,0.036578,0.197904,...,0.037164,0.059914,0.046520,0.039465,0.041603,0.038239,0.051552,0.038035,0.038569,0.042438
min,0.000000,-2.597617,-2.671613,-2.671613,-2.671613,-2.671613,-2.344165,-2.671613,-2.671613,-2.495544,...,-2.620769,-2.671613,-2.671613,-2.671613,-2.671613,-2.671613,-2.671613,-2.671613,-2.671613,-2.671613
25%,16320.250000,-2.209336,-2.220192,-2.223679,-2.211437,-2.192158,-2.154267,-2.218745,-2.219086,-2.197704,...,-2.219630,-2.216147,-2.220347,-2.221893,-2.218394,-2.220500,-2.221748,-2.224618,-2.224290,-2.216428
50%,32277.500000,-2.185969,-2.193247,-2.195208,-2.188057,-2.178255,-2.088598,-2.192513,-2.192915,-2.177229,...,-2.192232,-2.190525,-2.192814,-2.194127,-2.191564,-2.192200,-2.193352,-2.196185,-2.195900,-2.190953
75%,48467.750000,-2.175309,-2.179261,-2.179990,-2.176763,-2.153317,-1.975940,-2.178976,-2.179164,-2.123598,...,-2.178093,-2.177777,-2.178996,-2.179475,-2.178335,-2.177590,-2.178927,-2.180602,-2.180380,-2.178088
max,64993.000000,-1.596004,-1.341325,-1.917826,-1.814585,-1.607558,-1.209058,-1.474441,-1.879678,-0.977219,...,-1.855317,-0.960356,-1.357408,-1.775218,-1.790435,-1.799341,-1.335962,-1.816892,-1.718778,-1.500309


## Colinear?

In [98]:
train.cov()

,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,...,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
site_name,143.222099,-5.683001,117.986045,319.838418,-3.224373e+03,6.115038e+02,1.010036e+05,-0.029045,0.268109,-1.322880,...,-0.282185,0.086050,3.843352e+03,-0.275525,-0.033632,0.292368,3.938660,175.473016,-410.920508,-7.754511
posa_continent,-5.683001,0.559563,7.796804,-4.444136,4.873691e+02,5.787439e+01,-2.925821e+03,0.004165,-0.030434,0.255899,...,0.018453,-0.010853,-1.029484e+02,0.068593,0.001970,-0.013548,-0.405066,-6.571571,19.358628,0.323117
user_location_country,117.986045,7.796804,3509.745294,683.676340,1.227846e+05,5.820678e+03,-4.765141e+05,0.073091,-0.648602,23.351339,...,1.599158,-0.005766,5.949705e+03,3.897444,0.120725,0.199507,-6.467190,314.968489,552.137316,-17.948350
user_location_region,319.838418,-4.444136,683.676340,43448.796805,4.539165e+05,4.491614e+04,-1.094564e+05,1.187854,3.430112,0.467378,...,1.883036,0.025933,5.100098e+04,4.425131,0.358478,-2.260193,15.539729,-638.119113,4798.757178,44.924612
user_location_city,-3224.373297,487.369056,122784.598459,453916.549356,2.816541e+08,4.718788e+05,-4.044520e+07,8.257479,100.607673,1652.787203,...,110.997813,-0.525241,4.009083e+05,18.373580,10.122357,-24.592675,145.915305,-8857.848759,81128.518819,403.043824
orig_destination_distance,611.503839,57.874390,5820.678191,44916.144682,4.718788e+05,4.983799e+06,1.187211e+07,-42.158777,37.257370,24.572611,...,-103.515247,-7.339466,-6.577095e+05,-189.835644,-23.140756,26.567330,1386.739273,30561.048994,-92991.371826,472.206830
user_id,101003.558237,-2925.821039,-476514.108613,-109456.355427,-4.044520e+07,1.187211e+07,1.229326e+11,-688.740880,-1803.363524,-2659.477783,...,-56.805418,196.048071,1.168669e+07,3433.904385,172.785292,-162.638975,2148.430564,187014.172810,-735104.179905,10661.341821
is_mobile,-0.029045,0.004165,0.073091,1.187854,8.257479e+00,-4.215878e+01,-6.887409e+02,0.116721,0.008003,-0.041449,...,0.005107,-0.003512,-3.055361e+01,-0.013189,-0.002845,0.002059,-0.011713,-0.493535,1.367224,0.083097
is_package,0.268109,-0.030434,-0.648602,3.430112,1.006077e+02,3.725737e+01,-1.803364e+03,0.008003,0.186951,-0.016475,...,-0.010597,-0.007761,-7.059808e+02,-0.212441,-0.008938,0.066943,0.078483,-0.944996,-3.491601,0.484259
channel,-1.322880,0.255899,23.351339,0.467378,1.652787e+03,2.457261e+01,-2.659478e+03,-0.041449,-0.016475,13.816792,...,0.017763,0.011678,7.069519e+01,0.217257,0.024595,-0.054319,-0.128495,-0.400554,9.559863,0.075993


<font color='orange'>
    why are these cov numbers > 1? 
</font>

In [96]:
# code referenced from kj2546 hw1
cor_matrix = train.cov()
upper_triang = np.triu(np.ones(cor_matrix.shape), k=1).astype(np.bool)
to_consider = cor_matrix.where(upper_triang).abs()
drop_cols = [col for col in to_consider.columns if (to_consider[col] >= .9).sum() > 0]
train_wo_colinear_fts = train.drop(columns=drop_cols)
print(drop_cols)
train_wo_colinear_fts.head()

<ipython-input-96-cbab545ee2b4>:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper_triang = np.triu(np.ones(cor_matrix.shape), k=1).astype(np.bool)


['posa_continent', 'user_location_country', 'user_location_region', 'user_location_city', 'orig_destination_distance', 'user_id', 'is_mobile', 'is_package', 'channel', 'srch_adults_cnt', 'srch_children_cnt', 'srch_rm_cnt', 'srch_destination_id', 'srch_destination_type_id', 'is_booking', 'cnt', 'hotel_continent', 'hotel_country', 'hotel_market', 'hotel_cluster']


,date_time,site_name,srch_ci,srch_co
0,2014-08-11 07:46:59,2,2014-08-27,2014-08-31
1,2014-08-11 08:22:12,2,2014-08-29,2014-09-02
2,2014-08-11 08:24:33,2,2014-08-29,2014-09-02
3,2014-08-09 18:05:16,2,2014-11-23,2014-11-28
4,2014-08-09 18:08:18,2,2014-11-23,2014-11-28


## Cleaning
- how we handle NaNs (and explanations)
- should we remove user_id from data (and explanation)?